In [1]:
# 1. 필수 라이브러리 설치 및 임포트
# 모듈 설치 (Colab 환경에 mediapipe가 없다면 아래 주석을 풀고 실행하세요.)
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling opencv

In [1]:
import os
import cv2
import mediapipe as mp
import pandas as pd
import numpy as np
from google.colab import drive

# 2. 구글 드라이브 마운트
# 이 코드를 실행하면 구글 인증 절차가 진행됩니다.
drive.mount('/content/drive')

# 3. 동영상 파일 경로 및 추출할 시간 구간 설정
video_path = '/content/drive/MyDrive/2.mp4'
segment_name = 'shoulder_elbow_right'
start_time_ms = 127000
end_time_ms = 138000

Mounted at /content/drive


In [ ]:
# 4. Mediapipe Pose 모델 초기화
mp_pose = mp.solutions.pose
pose = mp.solutions.pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 5. 동영상 파일 처리 및 데이터 추출
print(f"\n> '{segment_name}' 구간 처리 시작: {start_time_ms/1000}초 ~ {end_time_ms/1000}초")

cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print(f"오류: '{os.path.basename(video_path)}' 파일을 열 수 없습니다.")
else:
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time_ms)

    all_landmarks_data = []
    frame_count = 0
    while cap.isOpened():
        if cap.get(cv2.CAP_PROP_POS_MSEC) > end_time_ms:
            break

        success, image = cap.read()
        if not success:
            break

        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks_row = {}
            # Clearly label landmarks with English names (관절별 라벨링)
            landmarks = results.pose_landmarks.landmark
            landmarks_to_extract = {
                'nose': landmarks[0],
                'left_shoulder': landmarks[11],
                'right_shoulder': landmarks[12],
                'left_elbow': landmarks[13],
                'right_elbow': landmarks[14],
                'left_wrist': landmarks[15],
                'right_wrist': landmarks[16],
                'left_hip': landmarks[23],
                'right_hip': landmarks[24]
            }

            for name, landmark in landmarks_to_extract.items():
                landmarks_row[f'{name}_x'] = landmark.x
                landmarks_row[f'{name}_y'] = landmark.y
                landmarks_row[f'{name}_z'] = landmark.z

            landmarks_row['frame'] = frame_count
            all_landmarks_data.append(landmarks_row)

        frame_count += 1

    cap.release()

    if all_landmarks_data:
        df = pd.DataFrame(all_landmarks_data)
        # 6. CSV 파일로 저장
        output_csv_path = f'/content/drive/MyDrive/stretch_pose_data_{segment_name}.csv'
        df.to_csv(output_csv_path, index=False)
        print(f"  > 추출 성공! 추출된 프레임 수: {len(df)}")
        print(f"  > 데이터가 '{output_csv_path}'에 저장되었습니다.")
    else:
        print("  > 오류: 지정된 시간대에 데이터를 추출하지 못했습니다.")

pose.close()

Mounted at /content/drive

> 'shoulder_elbow_right' 구간 처리 시작: 127.0초 ~ 138.0초
  > 추출 성공! 추출된 프레임 수: 332
  > 데이터가 '/content/drive/MyDrive/stretch_pose_data_shoulder_elbow_right.csv'에 저장되었습니다.


# 특징 공학 (Feature Engineering)

In [ ]:
# 최종 데이터셋 파일을 불러옵니다.
df = pd.read_csv(f'/content/drive/MyDrive/stretch_pose_data_{segment_name}.csv')

# --- 1. 거리 계산 (Distance Calculation) ---

# 각 관절 사이의 거리를 계산하는 함수를 정의합니다.
def calculate_distance(df, landmark1_name, landmark2_name):
    return np.sqrt(
        (df[f'{landmark1_name}_x'] - df[f'{landmark2_name}_x'])**2 +
        (df[f'{landmark1_name}_y'] - df[f'{landmark2_name}_y'])**2 +
        (df[f'{landmark1_name}_z'] - df[f'{landmark2_name}_z'])**2
    )

# 1. 왼손목(left_wrist)과 왼쪽 골반(left_hip) 사이의 거리
df['dist_left_wrist_hip'] = calculate_distance(df, 'left_wrist', 'left_hip')

# 2. 오른손목(right_wrist)과 오른쪽 골반(right_hip) 사이의 거리
df['dist_right_wrist_hip'] = calculate_distance(df, 'right_wrist', 'right_hip')

# 3. 양쪽 어깨(left_shoulder, right_shoulder) 사이의 거리
df['dist_shoulder_shoulder'] = calculate_distance(df, 'left_shoulder', 'right_shoulder')

# 4. 코(nose)와 양쪽 골반(hip)의 중간점 사이의 거리
df['dist_nose_hip_mid'] = np.sqrt(
    (df['nose_x'] - (df['left_hip_x'] + df['right_hip_x']) / 2)**2 +
    (df['nose_y'] - (df['left_hip_y'] + df['right_hip_y']) / 2)**2 +
    (df['nose_z'] - (df['left_hip_z'] + df['right_hip_z']) / 2)**2
)

# --- 2. 각도 계산 (Angle Calculation) ---

# 세 개의 점으로 각도를 계산하는 함수를 정의합니다.
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0)) # acos 인자 범위 클리핑

    return np.degrees(angle)

# 1. 왼쪽 팔꿈치 각도 계산 (왼쪽 어깨, 왼쪽 팔꿈치, 왼쪽 손목)
left_elbow_angles = []
for i in range(len(df)):
    angle = calculate_angle(
        (df.loc[i, 'left_shoulder_x':'left_shoulder_z'].values),
        (df.loc[i, 'left_elbow_x':'left_elbow_z'].values),
        (df.loc[i, 'left_wrist_x':'left_wrist_z'].values)
    )
    left_elbow_angles.append(angle)
df['angle_left_elbow'] = left_elbow_angles

# 2. 오른쪽 팔꿈치 각도 계산 (오른쪽 어깨, 오른쪽 팔꿈치, 오른쪽 손목)
right_elbow_angles = []
for i in range(len(df)):
    angle = calculate_angle(
        (df.loc[i, 'right_shoulder_x':'right_shoulder_z'].values),
        (df.loc[i, 'right_elbow_x':'right_elbow_z'].values),
        (df.loc[i, 'right_wrist_x':'right_wrist_z'].values)
    )
    right_elbow_angles.append(angle)
df['angle_right_elbow'] = right_elbow_angles

# --- 최종 파일 저장 ---

# 변경된 데이터프레임을 새로운 파일로 저장합니다.
output_path = f'/content/drive/MyDrive/{segment_name}_processed_features.csv'
df.to_csv(output_path, index=False)
print("\n모든 특징 공학 작업이 완료되었고, '{}' 파일이 저장되었습니다.".format(output_path))

# 최종 데이터셋의 일부를 확인합니다.
print("\n생성된 최종 데이터셋 미리보기:")
print(df.head())


모든 특징 공학 작업이 완료되었고, '/content/drive/MyDrive/shoulder_elbow_right_processed_features.csv' 파일이 저장되었습니다.

생성된 최종 데이터셋 미리보기:
     nose_x    nose_y    nose_z  left_shoulder_x  left_shoulder_y  \
0  0.464530  0.358208 -0.274831         0.551208         0.508045   
1  0.464170  0.355546 -0.297242         0.556611         0.508196   
2  0.463852  0.351481 -0.218618         0.557877         0.505644   
3  0.463567  0.351518 -0.227224         0.559277         0.504196   
4  0.463246  0.351559 -0.221471         0.559627         0.502593   

   left_shoulder_z  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0        -0.101919          0.447242          0.501159         -0.226080   
1        -0.076889          0.445456          0.500354         -0.205357   
2        -0.022819          0.444632          0.497623         -0.156148   
3        -0.014665          0.443672          0.496931         -0.159399   
4        -0.018640          0.443255          0.496330         -0.159293   

   lef

# 데이터 정규화(Data Normalization)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 특징이 추가된 데이터셋 파일을 불러옵니다.
df = pd.read_csv(f'/content/drive/MyDrive/{segment_name}_processed_features.csv')

# 정규화할 열들을 선택합니다.
# 'frame' 열은 정규화에서 제외합니다.
columns_to_normalize = df.columns.drop(['frame'])

# MinMaxScaler를 사용하여 데이터 정규화를 수행합니다.
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

# 정규화된 데이터셋 미리보기를 출력합니다.
print("정규화된 데이터셋 미리보기:")
print(df[columns_to_normalize].head())

# 정규화된 데이터셋을 새로운 파일로 저장합니다.
df.to_csv(f'/content/drive/MyDrive/{segment_name}_normalized.csv', index=False)

print(f"\n정규화된 데이터가 '{segment_name}_normalized.csv' 파일로 저장되었습니다.")

정규화된 데이터셋 미리보기:
     nose_x    nose_y    nose_z  left_shoulder_x  left_shoulder_y  \
0  1.000000  0.260403  0.168479         0.186642         0.927778   
1  0.980050  0.157364  0.000000         0.599775         0.943023   
2  0.962420  0.000000  0.591094         0.696576         0.685126   
3  0.946578  0.001436  0.526389         0.803691         0.538682   
4  0.928810  0.003027  0.569647         0.830437         0.376681   

   left_shoulder_z  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0         0.000000          1.000000          1.000000          0.000000   
1         0.276396          0.848929          0.951055          0.140437   
2         0.873458          0.779150          0.784994          0.473919   
3         0.963496          0.697956          0.742875          0.451885   
4         0.919602          0.662667          0.706383          0.452600   

   left_elbow_x  ...  left_hip_z  right_hip_x  right_hip_y  right_hip_z  \
0      0.424770  ...    1.000000     

# 라벨링

In [ ]:
# 정규화된 데이터셋 불러오기
df = pd.read_csv(f'/content/drive/MyDrive/{segment_name}_normalized.csv')

# 'label' 열을 추가하고 'neck_right'라는 값으로 채우기
df['label'] = f'{segment_name}'

# 라벨이 추가된 데이터셋 미리보기
print(df.head())

# 라벨이 추가된 파일을 새로운 이름으로 저장
df.to_csv(f'/content/drive/MyDrive/{segment_name}_labeled.csv', index=False)
print(f"\n'{segment_name}' 라벨링이 완료되었습니다.")

     nose_x    nose_y    nose_z  left_shoulder_x  left_shoulder_y  \
0  1.000000  0.260403  0.168479         0.186642         0.927778   
1  0.980050  0.157364  0.000000         0.599775         0.943023   
2  0.962420  0.000000  0.591094         0.696576         0.685126   
3  0.946578  0.001436  0.526389         0.803691         0.538682   
4  0.928810  0.003027  0.569647         0.830437         0.376681   

   left_shoulder_z  right_shoulder_x  right_shoulder_y  right_shoulder_z  \
0         0.000000          1.000000          1.000000          0.000000   
1         0.276396          0.848929          0.951055          0.140437   
2         0.873458          0.779150          0.784994          0.473919   
3         0.963496          0.697956          0.742875          0.451885   
4         0.919602          0.662667          0.706383          0.452600   

   left_elbow_x  ...  right_hip_y  right_hip_z  frame  dist_left_wrist_hip  \
0      0.424770  ...     0.000000     0.000000    

# 모델링 (오토인코더)

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.model_selection import train_test_split

# 올바른 동작 데이터만 불러옵니다.
# 'neck_left' 동작 데이터를 오토인코더에 학습시킵니다.
df = pd.read_csv(f'/content/drive/MyDrive/{segment_name}_labeled.csv')

# 'label' 열을 제거하고 입력 특징(features)을 선택합니다.
X = df.drop('label', axis=1).values

# 학습 데이터와 검증 데이터로 나눕니다.
X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

# ====== 오토인코더 모델 구조 정의 ======
# 입력 레이어
input_layer = Input(shape=(X_train.shape[1],))
# 인코더 (데이터 압축)
encoder = Dense(64, activation='relu')(input_layer)
encoder = Dense(32, activation='relu')(encoder)
encoder = Dense(16, activation='relu')(encoder)
# 디코더 (데이터 복원)
decoder = Dense(32, activation='relu')(encoder)
decoder = Dense(64, activation='relu')(decoder)
decoder = Dense(X_train.shape[1], activation='linear')(decoder)

# 오토인코더 모델 생성
autoencoder = Model(inputs=input_layer, outputs=decoder)

# 모델 컴파일 (오차를 최소화하도록 학습시킵니다)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
history = autoencoder.fit(X_train, X_train,
                          epochs=50,
                          batch_size=256,
                          shuffle=True,
                          validation_data=(X_test, X_test))

# 학습된 모델을 저장합니다.
autoencoder.save(f'/content/drive/MyDrive/{segment_name}_autoencoder_model.h5')

print(f"\n\n모델 학습이 완료되었고, '{segment_name}_autoencoder_model.h5' 파일로 저장되었습니다!")

Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - loss: 1114.3746 - val_loss: 986.3467
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1079.3411 - val_loss: 961.4736
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1054.1549 - val_loss: 941.9092
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1032.3209 - val_loss: 925.8646
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 1015.9083 - val_loss: 908.8077
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 999.7075 - val_loss: 888.8294
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 975.8585 - val_loss: 864.2592
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 943.5963 - val_loss: 834.8885
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 917.6243 - val_loss: 799.3387
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 878.5176 - val_loss: 759.6201
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 836.4135 - val_loss: 716.6299
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0



모델 학습이 완료되었고, 'shoulder_elbow_right_autoencoder_model.h5' 파일로 저장되었습니다!
